%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j wikipedia tiktoken yfiles_jupyter_graphs

In [9]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_community.document_loaders import DirectoryLoader, OnlinePDFLoader, PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Tuple, List, Optional
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_community.graphs import Neo4jGraph
from langchain.document_loaders import WikipediaLoader
from langchain.text_splitter import TokenTextSplitter
from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_core.runnables import ConfigurableField, RunnableParallel, RunnablePassthrough


"""
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass
"""

'\ntry:\n  import google.colab\n  from google.colab import output\n  output.enable_custom_widget_manager()\nexcept:\n  pass\n'

# Enhancing RAG-based applications accuracy by constructing and leveraging knowledge graphs
## A practical guide to constructing and retrieving information from knowledge graphs in RAG applications with Neo4j and LangChain

Graph retrieval augmented generation (Graph RAG) is gaining momentum and emerging as a powerful addition to traditional vector search retrieval methods. This approach leverages the structured nature of graph databases, which organize data as nodes and relationships, to enhance the depth and contextuality of retrieved information.

Graphs are great at representing and storing heterogeneous and interconnected information in a structured manner, effortlessly capturing complex relationships and attributes across diverse data types. In contrast, vector databases often struggle with such structured information, as their strength lies in handling unstructured data through high-dimensional vectors. In your RAG application, you can combine structured graph data with vector search through unstructured text to achieve the best of both worlds, which is exactly what we will do in this blog post.

Knowledge graphs are great, but how do you create one? Constructing a knowledge graph is typically the most challenging step in leveraging the power of graph-based data representation. It involves gathering and structuring the data, which requires a deep understanding of both the domain and graph modeling. To simplify this process, we have been experimenting with LLMs. LLMs, with their profound understanding of language and context, can automate significant parts of the knowledge graph creation process. By analyzing text data, these models can identify entities, understand the relationships between them, and suggest how they might be best represented in a graph structure. As a result of these experiments, we have added the first version of the graph construction module to LangChain, which we will demonstrate in this blog post.

## Neo4j Environment Setup

You need to set up a Neo4j instance follow along with the examples in this blog post. The easiest way is to start a free instance on [Neo4j Aura](https://neo4j.com/cloud/platform/aura-graph-database/), which offers cloud instances of Neo4j database. Alternatively, you can also set up a local instance of the Neo4j database by downloading the Neo4j Desktop application and creating a local database instance.

In [28]:
# so.api pin here
graph = Neo4jGraph()

In [29]:
# Read the wikipedia article
#raw_documents = WikipediaLoader(query="Elizabeth I").load()
# Define chunking strategy
#text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
#documents = text_splitter.split_documents(raw_documents[:3])

pdf_folder = ""

#loader = DirectoryLoader(pdf_folder, glob="SPE5/*.pdf", loader_cls=PyPDFLoader)
loader = DirectoryLoader(pdf_folder, glob="abs/*.pdf", loader_cls=PyPDFLoader)


raw_documents = loader.load()
# Define chunking strategy
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(raw_documents) 

In [30]:
print(documents)

[Document(metadata={'source': 'abs\\1-mvr-part-4-jul24_432.pdf', 'page': 0}, page_content='C H A P T E R  3\nPropulsion and Maneuvering Machinery\nS E C T I O N  2\nPropulsion Shafting\n1 General\n1.1 Application\nThis section applies to shafts, couplings, clutches and other power transmitting components for propulsion\npurposes.\nShafts and associated components used for transmission of power, essential for the propulsion of the\nvessel, are to be so designed and constructed to withstand the maximum working stresses to which they\nmay be subjected in all service conditions.\nConsideration may be given to designs based on engineering analyses, including fatigue considerations, as\nan alternative to the requirements of this section. Alternative calculation methods are to take into account\ndesign criteria for continuous and transient operating loads (dimensioning for fatigue strength) and for\npeak operating loads (dimensioning for yield strength).\nNotes:\nShafts complying with this se

Now it's time to construct a graph based on the retrieved documents. For this purpose, we have implemented an `LLMGraphTransformermodule` that significantly simplifies constructing and storing a knowledge graph in a graph database.

In [13]:
llm=ChatOpenAI(temperature=0, model_name="gpt-4o") # gpt-4-0125-preview occasionally has issues
llm_transformer = LLMGraphTransformer(llm=llm)

graph_documents = llm_transformer.convert_to_graph_documents(documents)
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

You can define which LLM you want the knowledge graph generation chain to use. At the moment, we support only function calling models from OpenAI and Mistral. However, we plan to expand the LLM selection in the future. In this example, we are using the latest GPT-4. Note that the quality of generated graph significantly depends on the model you are using. In theory, you always want to use the most capable one. 

The LLM graph transformers returns graph documents, which can be imported to Neo4j via the `add_graph_documents` method. 
The `baseEntityLabel` parameter assigns an additional `__Entity__` label to each node, enhancing indexing and query performance. 
The `include_source` parameter links nodes to their originating documents, facilitating data traceability and context understanding.

You can inspect the generated graph with yfiles visualization.

In [40]:
# directly show the graph resulting from the given Cypher query
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    #display(widget)     ##
    return widget

showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

## Hybrid Retrieval for RAG    *******key
After the graph generation, we will use a hybrid retrieval approach that combines vector and keyword indexes with graph retrieval for RAG applications.

![retrieval](https://raw.githubusercontent.com/tomasonjo/blogs/master/graphhybrid.png)

The diagram illustrates a retrieval process beginning with a user posing a question, which is then directed to an RAG retriever. This retriever employs keyword and vector searches to search through unstructured text data and combines it with the information it collects from the knowledge graph. Since Neo4j features both keyword and vector indexes, you can implement all three retrieval options with a single database system. The collected data from these sources is fed into an LLM to generate and deliver the final answer.
## Unstructured data retriever
You can use the Neo4jVector.from_existing_graph method to add both keyword and vector retrieval to documents. This method configures keyword and vector search indexes for a hybrid search approach, targeting nodes labeled Document. Additionally, it calculates text embedding values if they are missing.



In [15]:
vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    search_type="hybrid",              ######
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated procedure. ('db.index.vector.createNodeIndex' has been replaced by 'CREATE VECTOR INDEX')} {position: line: 1, column: 1, offset: 0} for query: 'CALL db.index.vector.createNodeIndex($index_name,$node_label,$embedding_node_property,toInteger($embedding_dimension),$similarity_metric )'


The vector index can then be called with the similarity_search method.
## Graph retriever
On the other hand, configuring a graph retrieval is more involved but offers more freedom. In this example, we will use a full-text index to identify relevant nodes and then return their direct neighborhood.

![graph](https://raw.githubusercontent.com/tomasonjo/blogs/master/neighbor.png)

The graph retriever starts by identifying relevant entities in the input. 
For simplicity, we instruct the LLM to identify people, organizations, and locations. 

To achieve this, we will use LCEL with the newly added `with_structured_output` method to achieve this.




LCEL   #####




In [16]:
# Retriever

graph.query(
    "CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

entity_chain = prompt | llm.with_structured_output(Entities)

Let's test it out:

In [44]:
entity_chain.invoke({"question": "what shafting includes"}).names

[]

Great, now that we can detect entities in the question, let's use a full-text index to map them to the knowledge graph. First, we need to define a full-text index and a function that will generate full-text queries that allow a bit of misspelling, which we won't go into much detail here.

In [18]:
def generate_full_text_query(input: str) -> str:
    """
    Generate a full-text search query for a given input string.

    This function constructs a query string suitable for a full-text search.
    It processes the input string by splitting it into words and appending a
    similarity threshold (~2 changed characters) to each word, then combines
    them using the AND operator. Useful for mapping entities from user questions
    to database values, and allows for some misspelings.
    """
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

# Fulltext index query
def structured_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

The `structured_retriever` function starts by detecting entities in the user question. Next, it iterates over the detected entities and uses a Cypher template to retrieve the neighborhood of relevant nodes. Let's test it out!

In [32]:
print(structured_retriever("What is shaft?"))

## Final retriever
As we mentioned at the start, we'll combine the unstructured and graph retriever to create the final context that will be passed to an LLM.

In [20]:
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    return final_data

As we are dealing with Python, we can simply concatenate the outputs using the f-string.
## Defining the RAG chain
We have successfully implemented the retrieval component of the RAG. First, we will introduce the query rewriting part that allows conversational follow up questions.


In [21]:
# Condense a chat history and follow-up question into a standalone question
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""  # noqa: E501
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatOpenAI(temperature=0)
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x : x["question"]),
)

Next, we introduce a prompt that leverages the context provided by the integrated hybrid retriever to produce the response, completing the implementation of the RAG chain.

In [22]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

Finally, we can go ahead and test our hybrid RAG implementation.

In [23]:
chain.invoke({"question": "how to design propulsion shaft??"})

Search query: how to improve unconventional production??


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL () { ... }} {position: line: 1, column: 1, offset: 0} for query: "CALL { CALL db.index.vector.queryNodes($index, $k, $embedding) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score UNION CALL db.index.fulltext.queryNodes($keyword_index, $query, {limit: $k}) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score } WITH node, max(score) AS score ORDER BY score DESC LIMIT $k RETURN reduce(str='', k IN ['text'] | str + '\\n' + k + ': ' + coalesce(node[k], '')) AS text, node {.*, `embedding

'The provided context does not contain information related to improving unconventional production.'

Let's test a follow up question!

In [24]:
chain.invoke(
    {
        "question": "When was she born?",
        "chat_history": [("Which house did Elizabeth I belong to?", "House Of Tudor")],
    }
)

Search query: When was Elizabeth I born?


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 13, offset: 104} for query: "CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            }\n            RETURN output LIMIT 50\n            "
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNoti

"The context provided does not contain information about Elizabeth I's birth date."

In [35]:
chain.invoke({"question": "how to design shaft alignment for vessels"})

Search query: how to design shaft alignment for vessels


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL () { ... }} {position: line: 1, column: 1, offset: 0} for query: "CALL { CALL db.index.vector.queryNodes($index, $k, $embedding) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score UNION CALL db.index.fulltext.queryNodes($keyword_index, $query, {limit: $k}) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score } WITH node, max(score) AS score ORDER BY score DESC LIMIT $k RETURN reduce(str='', k IN ['text'] | str + '\\n' + k + ': ' + coalesce(node[k], '')) AS text, node {.*, `embedding

'To design shaft alignment for vessels, submit shaft alignment calculations, alignment procedures, and stern tube boring details for review. Ensure the propulsion shafting has a diameter greater than 300 mm or lacks a forward stern tube bearing. The alignment procedure should include bore sighting before and after fitting bearings to verify dimensions, misalignment, and slopes. Minimize horizontal misalignment and verify slope boring angles relative to the centerline connecting bearings. Adjustments should be made by machining the outside bush diameter when applicable.'

In [38]:
chain.invoke({"question": "how to better design bolts? what are the most important considerations? what are the detail requirements for each concern, plist list the spedific values if any"})

Search query: how to better design bolts? what are the most important considerations? what are the detail requirements for each concern, plist list the spedific values if any


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL () { ... }} {position: line: 1, column: 1, offset: 0} for query: "CALL { CALL db.index.vector.queryNodes($index, $k, $embedding) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score UNION CALL db.index.fulltext.queryNodes($keyword_index, $query, {limit: $k}) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score } WITH node, max(score) AS score ORDER BY score DESC LIMIT $k RETURN reduce(str='', k IN ['text'] | str + '\\n' + k + ': ' + coalesce(node[k], '')) AS text, node {.*, `embedding

'To better design bolts, the most important considerations include:\n\n1. **Tensile Stress**: The tensile stress of the bolt due to pre-stressing and astern pull should not exceed 90% of the minimum specified yield strength of the bolt material.\n\n2. **Bearing Stress**: The bearing stress on any member such as the flange, bolt head, threads, or nut should not exceed 90% of the minimum specified yield strength of the material of that member.\n\n3. **Loosening Prevention**: Bolts should be provided with means to prevent loosening in service.\n\nThese considerations ensure the structural integrity and reliability of the bolts in their application.'

In [34]:
chain.invoke({"question": "how to better design bolts? what are the most important considerations"})

Search query: how to better design bolts? what are the most important considerations


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL () { ... }} {position: line: 1, column: 1, offset: 0} for query: "CALL { CALL db.index.vector.queryNodes($index, $k, $embedding) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score UNION CALL db.index.fulltext.queryNodes($keyword_index, $query, {limit: $k}) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score } WITH node, max(score) AS score ORDER BY score DESC LIMIT $k RETURN reduce(str='', k IN ['text'] | str + '\\n' + k + ': ' + coalesce(node[k], '')) AS text, node {.*, `embedding

'To better design bolts, the most important considerations include ensuring an accurate fit, selecting materials with appropriate tensile strength, and considering the torque transmission method. For fitted bolts, ensure an interference fit and calculate the minimum diameter using specified equations. For non-fitted bolts, focus on detailed preloading and stress calculations, ensuring tensile stress does not exceed 90% of the yield strength. Additionally, consider the factor of safety against slip, especially under worst operating conditions, and ensure the bearing stress on components like flanges and threads does not exceed material yield strength.'

In [39]:
chain.invoke({"question": "how many kinds of bolts are there, and what are the specific definitions and design concerns for each kind?"})

Search query: how many kinds of bolts are there, and what are the specific definitions and design concerns for each kind?


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL () { ... }} {position: line: 1, column: 1, offset: 0} for query: "CALL { CALL db.index.vector.queryNodes($index, $k, $embedding) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score UNION CALL db.index.fulltext.queryNodes($keyword_index, $query, {limit: $k}) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score } WITH node, max(score) AS score ORDER BY score DESC LIMIT $k RETURN reduce(str='', k IN ['text'] | str + '\\n' + k + ': ' + coalesce(node[k], '')) AS text, node {.*, `embedding

'There are two kinds of bolts mentioned: coupling bolts and non-fitted bolts. \n\n1. **Coupling Bolts**: These can be integral, demountable, keyed, or shrink-fit. Specific details regarding the interference fit of the coupling bolts must be submitted, along with calculations and design basis for sizing if not based on the as-built line shaft diameter.\n\n2. **Non-fitted Bolts**: The tensile stress due to pre-stressing and astern pull should not exceed 90% of the minimum specified yield strength of the bolt material. Bearing stress on any member like the flange, bolt head, threads, or nut should also not exceed 90% of the yield strength of the material. These bolts must have means to prevent loosening in service.'

In [46]:
chain.invoke({"question": "what are requirements for oil lubricated bearings? give the specfic values please"})

Search query: what are requirements for oil lubricated bearings? give the specfic values please


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL () { ... }} {position: line: 1, column: 1, offset: 0} for query: "CALL { CALL db.index.vector.queryNodes($index, $k, $embedding) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score UNION CALL db.index.fulltext.queryNodes($keyword_index, $query, {limit: $k}) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score } WITH node, max(score) AS score ORDER BY score DESC LIMIT $k RETURN reduce(str='', k IN ['text'] | str + '\\n' + k + ': ' + coalesce(node[k], '')) AS text, node {.*, `embedding

'For oil-lubricated bearings, the requirements are as follows:\n\n1. **White Metal Bearings**: The length must be at least two times the required tail shaft diameter. It can be reduced if the nominal bearing pressure does not exceed 0.80 N/mm² (0.0815 kgf/mm², 116 psi). The minimum length should not be less than 1.5 times the actual diameter.\n\n2. **Synthetic Material Bearings**: The length must be at least two times the required tail shaft diameter. It can be reduced if the nominal bearing pressure does not exceed 0.60 N/mm² (0.0611 kgf/mm², 87 psi). The minimum length should not be less than 1.5 times the actual diameter. If the pressure exceeds 0.60 N/mm², test data and satisfactory service experience must be submitted.'

In [ ]:
'For oil-lubricated bearings, the requirements are as follows:
\n\n1. **White Metal Bearings**: The length must be at least two times the required tail shaft diameter. It can be reduced if the nominal bearing pressure does not exceed 0.80 N/mm² (0.0815 kgf/mm², 116 psi). The minimum length should not be less than 1.5 times the actual diameter.
\n\n2. **Synthetic Material Bearings**: The length must be at least two times the required tail shaft diameter. It can be reduced if the nominal bearing pressure does not exceed 0.60 N/mm² (0.0611 kgf/mm², 87 psi). The minimum length should not be less than 1.5 times the actual diameter. If the pressure exceeds 0.60 N/mm², test data and satisfactory service experience must be submitted.'


In [47]:
chain.invoke({"question": "what are the specric requirements of oil lubricated bearings"})

Search query: what are the specric requirements of oil lubricated bearings


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL () { ... }} {position: line: 1, column: 1, offset: 0} for query: "CALL { CALL db.index.vector.queryNodes($index, $k, $embedding) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score UNION CALL db.index.fulltext.queryNodes($keyword_index, $query, {limit: $k}) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score } WITH node, max(score) AS score ORDER BY score DESC LIMIT $k RETURN reduce(str='', k IN ['text'] | str + '\\n' + k + ': ' + coalesce(node[k], '')) AS text, node {.*, `embedding

'Oil-lubricated bearings have specific requirements based on the material used. For white metal bearings, the length must be at least two times the tail shaft diameter, with a possible reduction if the nominal bearing pressure does not exceed 0.80 N/mm². For synthetic materials, the length must be at least two times the tail shaft diameter, with a possible reduction if the nominal bearing pressure does not exceed 0.60 N/mm². In both cases, the minimum length cannot be less than 1.5 times the actual diameter. Additionally, an arrangement for accurate oil sampling and contaminant removal is required, and environmentally acceptable lubricants (EALs) must be compatible with shaft seal elastomer materials.'

In [48]:
chain.invoke({"question": "how many kind of viberation we need to consider? what are the major concerns and requirement paremeter value for each kind?"})

Search query: how many kind of viberation we need to consider? what are the major concerns and requirement paremeter value for each kind?


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL () { ... }} {position: line: 1, column: 1, offset: 0} for query: "CALL { CALL db.index.vector.queryNodes($index, $k, $embedding) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score UNION CALL db.index.fulltext.queryNodes($keyword_index, $query, {limit: $k}) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score } WITH node, max(score) AS score ORDER BY score DESC LIMIT $k RETURN reduce(str='', k IN ['text'] | str + '\\n' + k + ': ' + coalesce(node[k], '')) AS text, node {.*, `embedding

'There are three kinds of vibrations to consider: whirling vibrations, axial vibrations, and torsional vibrations.\n\n1. **Whirling Vibrations**: \n   - Major concerns include ensuring satisfactory vibration characteristics throughout the speed range and identifying critical speeds that may result in significant vibration amplitudes.\n   - Calculations must consider bearing and oil-film stiffness and investigate excitation frequencies.\n   - If critical speeds are within ±20% of maximum continuous ratings (M.C.R.) speed, measurements are required to determine the need for barred speed ranges.\n\n2. **Axial Vibrations**:\n   - Barred speed ranges identified in calculations must be verified and recorded through appropriate measurement procedures in the presence of a Surveyor.\n\n3. **Torsional Vibrations**:\n   - Torsiograph measurements are required in the presence of a Surveyor.\n   - If a barred speed range is provided, tachometer marking, warning notices, and alarms at remote control

In [49]:
chain.invoke({"question": "how many material requirements are specified in this documentation? and what are the detailed requirements for each of them"})

Search query: how many material requirements are specified in this documentation? and what are the detailed requirements for each of them


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL () { ... }} {position: line: 1, column: 1, offset: 0} for query: "CALL { CALL db.index.vector.queryNodes($index, $k, $embedding) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score UNION CALL db.index.fulltext.queryNodes($keyword_index, $query, {limit: $k}) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score } WITH node, max(score) AS score ORDER BY score DESC LIMIT $k RETURN reduce(str='', k IN ['text'] | str + '\\n' + k + ': ' + coalesce(node[k], '')) AS text, node {.*, `embedding

"The documentation specifies several material requirements, particularly for torque-transmitting parts and propulsion shafting. Here are the detailed requirements:\n\n1. **General Material Requirements:**\n   - Materials for propulsion shafts, couplings, coupling bolts, keys, and clutches must be of forged steel or rolled bars.\n   - They must comply with Sections 2-3-7 and 2-3-8 of the ABS Rules for Materials and Welding (Part 2).\n   - Alternative material specifications require special approval with full details of chemical composition, heat treatment, and mechanical properties.\n\n2. **Material Tests:**\n   - Materials for all torque-transmitting parts must be tested in the presence of a Surveyor.\n   - For parts transmitting 375 kW (500 hp) or less, verification of manufacturer's certification and a witnessed hardness check by the Surveyor are acceptable.\n   - Coupling bolts manufactured to a recognized standard do not require material testing.\n\n3. **Inspections and Nondestruct

In [ ]:
"The documentation specifies several material requirements, particularly for torque-transmitting parts and propulsion shafting. Here are the detailed requirements:
\n\n1. **General Material Requirements:**
\n   - Materials for propulsion shafts, couplings, coupling bolts, keys, and clutches must be of forged steel or rolled bars.
\n   - They must comply with Sections 2-3-7 and 2-3-8 of the ABS Rules for Materials and Welding (Part 2).
\n   - Alternative material specifications require special approval with full details of chemical composition, heat treatment, and mechanical properties.
\n\n2. **Material Tests:
**\ n   - Materials for all torque-transmitting parts must be tested in the presence of a Surveyor.
\n   - For parts transmitting 375 kW (500 hp) or less, verification of manufacturer's certification and a witnessed hardness check by the Surveyor are acceptable.
\n   - Coupling bolts manufactured to a recognized standard do not require material testing.\n\n3. **Inspections and Nondestructive Tests:
**\n   - Shafting and couplings must be surface examined by the Surveyor.\n   - Forgings for tail shafts 455 mm (18 in.) and over must be ultrasonically examined.
\n   - Tail shafts in finished condition must undergo magnetic particle, dye penetrant, or other non-destructive examinations and be free of significant linear discontinuities.
\n\n4. **Tailshaft Condition Monitoring (TCM-W):**\n   - Bearing material must be approved by ABS.
\n   - Corrosion-resistant material or coating is required for components exposed to seawater.
\n\nThese requirements ensure the materials used in marine vessel construction meet safety and performance standards."

In [50]:
chain.invoke({"question": "what are the major points of this documentation? list them in detail one by one, please"})

Search query: what are the major points of this documentation? list them in detail one by one, please


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL () { ... }} {position: line: 1, column: 1, offset: 0} for query: "CALL { CALL db.index.vector.queryNodes($index, $k, $embedding) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score UNION CALL db.index.fulltext.queryNodes($keyword_index, $query, {limit: $k}) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score } WITH node, max(score) AS score ORDER BY score DESC LIMIT $k RETURN reduce(str='', k IN ['text'] | str + '\\n' + k + ': ' + coalesce(node[k], '')) AS text, node {.*, `embedding

'1. **Emergency Lubrication Supply**: An emergency supply of lubricating water is required in case of failure of the primary lubrication system.\n\n2. **Lubricant Tank Specifications**: Tanks should be metallic or non-metallic, following recognized standards. Specifications for thermal, mechanical, chemical, and fire resistance must be submitted for review. Mounting, securing, and electrical bonding arrangements need approval. Valves should be accessible and operable manually if power fails.\n\n3. **Water Filtration System**: Two independent systems are required for continuous operation, with an auto change-over system in case of failure. Failures should trigger alarms, and normal conditions should be displayed.\n\n4. **Instrumentation and Alarms**: Instruments for monitoring the water-lubricated stern tube system must be provided. Alarms should be audible, visual, and self-monitoring, with testing provisions.\n\n5. **Vessel Conditions for Analysis**: Consider drydock, full ballast, an

In [51]:
chain.invoke({"question": "summrize this documentation"})

Search query: summrize this documentation


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL () { ... }} {position: line: 1, column: 1, offset: 0} for query: "CALL { CALL db.index.vector.queryNodes($index, $k, $embedding) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score UNION CALL db.index.fulltext.queryNodes($keyword_index, $query, {limit: $k}) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score } WITH node, max(score) AS score ORDER BY score DESC LIMIT $k RETURN reduce(str='', k IN ['text'] | str + '\\n' + k + ': ' + coalesce(node[k], '')) AS text, node {.*, `embedding

'The documentation outlines the requirements and procedures for vessels with water-lubricated bearings, focusing on compliance with specific standards. Key aspects include the need for a stern inspection chamber, installation of a split type aftmost bearing with appropriate seals, and a seawater cooling system with redundancy. It also covers the maintenance of records for oil condition and replacement, anti-freeze properties of lubricant water, and alignment verification in the presence of a Surveyor. Additionally, it specifies the run-in procedure for certain shaft installations.'